In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Load the Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

train = torch.load("/content/drive/MyDrive/Colab Notebooks/DL1/train.pt")
valid = torch.load("/content/drive/MyDrive/Colab Notebooks/DL1/val.pt")
test = torch.load("/content/drive/MyDrive/Colab Notebooks/DL1/test.pt")

<ipython-input-2-362843f868f5>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train = torch.load("/content/drive/MyDrive/Colab Notebooks/DL1/train.pt")
<ipython-input-2-36

In [ ]:
#Knowing the shapes of dataset
print("Shape of tensor_data train:", train['samples'].shape)
print("Shape of tensor_data valid:", valid['samples'].shape)
print("Shape of tensor_data test:", test['samples'].shape)

Shape of tensor_data train: torch.Size([5881, 3, 206])
Shape of tensor_data valid: torch.Size([1471, 3, 206])
Shape of tensor_data test: torch.Size([2947, 3, 206])


In [ ]:
'''1. Dimension 1: 5881 (train), 1471 (valid), 2947 (test)

This dimension represents the number of samples or data points in each of your datasets
(training, validation, and testing).

There are 5881 samples in the training dataset.
There are 1471 samples in the validation dataset.
There are 2947 samples in the testing dataset.

2. Dimension 2: 3
this dimension most likely represents the 3-axis accelerometer data.
Accelerometers typically measure acceleration in three directions:

X-axis: Movement along the horizontal axis.
Y-axis: Movement along the vertical axis.
Z-axis: Movement along the depth axis.

3. Dimension 3: 206

This dimension represents the number of time steps or readings within a single sample or window of activity.
This could be interpreted as a time window of 206 sensor samples'''

'1. Dimension 1: 5881 (train), 1471 (valid), 2947 (test)\n\nThis dimension represents the number of samples or data points in each of your datasets \n(training, validation, and testing).\n\nThere are 5881 samples in the training dataset.\nThere are 1471 samples in the validation dataset.\nThere are 2947 samples in the testing dataset.\n\n2. Dimension 2: 3\nthis dimension most likely represents the 3-axis accelerometer data.\nAccelerometers typically measure acceleration in three directions:\n\nX-axis: Movement along the horizontal axis.\nY-axis: Movement along the vertical axis.\nZ-axis: Movement along the depth axis.\n\n3. Dimension 3: 206\n\nThis dimension represents the number of time steps or readings within a single sample or window of activity.\nThis could be interpreted as a time window of 206 sensor samples'

In [ ]:
# Extract samples and labels
train_samples, train_labels = train['samples'], train['labels']
valid_samples, valid_labels = valid['samples'], valid['labels']
test_samples, test_labels = test['samples'], test['labels']

# Create datasets and dataloaders
train_dataset = TensorDataset(train_samples, train_labels)
valid_dataset = TensorDataset(valid_samples, valid_labels)
test_dataset = TensorDataset(test_samples, test_labels)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
# Define the RNN model
class HAR_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(HAR_RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
# Instantiate the model, optimizer, and loss function
input_size = 206  # 3 accelerometer channels
hidden_size = 128
num_layers = 2
num_classes = 6  # 6 activity classes

model = HAR_RNN(input_size, hidden_size, num_layers, num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 35
best_valid_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        target = target.type(torch.LongTensor)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in valid_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    valid_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {valid_acc:.2f}%")

    # Save best model checkpoint
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), 'best_model.pth')

# Load best model and evaluate on test set
model.load_state_dict(torch.load('best_model.pth', map_location=torch.device('cpu'),weights_only=True), strict=False)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)  # Call torch.max with output and unpack
        total += target.size(0)
        correct += (predicted == target).sum().item()

test_acc = 100 * correct / total
print(f"Test Accuracy: {test_acc:.2f}%")

Epoch [1/35], Validation Accuracy: 53.30%
Epoch [2/35], Validation Accuracy: 61.05%
Epoch [3/35], Validation Accuracy: 59.01%
Epoch [4/35], Validation Accuracy: 58.33%
Epoch [5/35], Validation Accuracy: 61.05%
Epoch [6/35], Validation Accuracy: 61.59%
Epoch [7/35], Validation Accuracy: 63.29%
Epoch [8/35], Validation Accuracy: 77.02%
Epoch [9/35], Validation Accuracy: 69.41%
Epoch [10/35], Validation Accuracy: 85.32%
Epoch [11/35], Validation Accuracy: 87.97%
Epoch [12/35], Validation Accuracy: 87.56%
Epoch [13/35], Validation Accuracy: 88.38%
Epoch [14/35], Validation Accuracy: 89.67%
Epoch [15/35], Validation Accuracy: 90.01%
Epoch [16/35], Validation Accuracy: 89.94%
Epoch [17/35], Validation Accuracy: 89.12%
Epoch [18/35], Validation Accuracy: 91.64%
Epoch [19/35], Validation Accuracy: 90.96%
Epoch [20/35], Validation Accuracy: 91.98%
Epoch [21/35], Validation Accuracy: 91.91%
Epoch [22/35], Validation Accuracy: 92.25%
Epoch [23/35], Validation Accuracy: 92.39%
Epoch [24/35], Valid